In [ ]:
#Libraries import 
import cv2
import datetime as dt
import math
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
import optuna
import os
import pandas as pd
import pickle
import random
import rarfile
import seaborn as sns
import ssl
import time
import urllib
import urllib.request
from collections import defaultdict, deque
from IPython.display import Video
from keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, LSTM, MaxPooling2D, TimeDistributed
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.utils import to_categorical
from moviepy.editor import *
from sklearn.metrics import auc, classification_report, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from tensorflow import keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.layers import *
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model, to_categorical
from tqdm import tqdm
from ultralytics import YOLO
from ultralytics.utils.ops import non_max_suppression
from urllib.request import urlretrieve
from yt_dlp import YoutubeDL
import yt_dlp as youtube_dl


In [ ]:
#Setting constant seed
seed_constant = 27

# Set the seed for NumPy random number generator to 'seed_constant'.
np.random.seed(seed_constant)

# Set the seed for Python's built-in random number generator to 'seed_constant'.
random.seed(seed_constant)

# Set the seed for TensorFlow's random number generator to 'seed_constant'.
tf.random.set_seed(seed_constant)


In [ ]:
#Dataset Download

# Define the URL of the UCF50 dataset.
DATA_URL = 'https://www.crcv.ucf.edu/data/UCF50.rar'

# Define the local directory path where the dataset will be downloaded.
DATA_PATH = 'workspace'

# Create the complete path to the directory where the UCF50 dataset will be stored after extraction.
UCF50_DATA_PATH = os.path.join(DATA_PATH, 'UCF50')

# Check if the directory specified by DATA_PATH already exists.
if os.path.exists(DATA_PATH):

    # If the directory exists, print a message indicating that the data is already available.
    print('[INFO] Data already exists.')

else:

    # If the directory specified by DATA_PATH does not exist, execute the following block.

    # Print a message indicating that the data is being downloaded.
    print('[INFO] Downloading data in the data directory.')

    # Create the DATA_PATH directory on the local file system.
    os.mkdir(DATA_PATH)

    # Create a default SSL context with an unverified SSL certificate to allow downloading data.
    ssl._create_default_https_context = ssl._create_unverified_context

    # Download the UCF50 dataset from the specified DATA_URL and save it as 'UCF50.rar' in the DATA_PATH directory.
    urlretrieve(url=DATA_URL, filename=os.path.join(DATA_PATH, 'UCF50.rar'))


In [ ]:
#Dataset Extraction 

# Check if the directory specified by UCF50_DATA_PATH already exists.
if os.path.exists(UCF50_DATA_PATH):

    # If the directory exists, print a message indicating that the data is already available, and the extraction process is skipped.
    print('[INFO] UCF50 Data already exists, skipping extraction process.')

else:

    # If the directory specified by UCF50_DATA_PATH does not exist, execute the following block.

    # Print a message indicating that the data is being extracted to the UCF50_DATA_PATH directory.
    print(f'[INFO] Extracting data: "{UCF50_DATA_PATH}"')

    # Create a RarFile object 'r' to open and read the 'UCF50.rar' archive file.
    r = rarfile.RarFile('/workspace/Workspace/UCF50.rar')

    # Extract all files and directories from the 'UCF50.rar' archive to the DATA_PATH directory.
    r.extractall(DATA_PATH)

    # Close the RarFile object to release resources.
    r.close()

In [ ]:
# Dataset Preview
# Create a Matplotlib figure with a specified size of 20x20 inches.
plt.figure(figsize=(20, 20))

# Get the list of all class names in the UCF50 directory.
all_classes_names = os.listdir('workspace/UCF50')

# Randomly select 10 class names from the list of all classes.
random_range = random.sample(range(len(all_classes_names)), 10)

# Iterate through the randomly selected class indices and their corresponding counter.
for counter, random_index in enumerate(random_range, 1):

    # Get the selected class name using the random index.
    selected_class_Name = all_classes_names[random_index]

    # Get the list of video file names inside the selected class directory.
    #video_files_names_list = os.listdir(f'UCF50/{selected_class_Name}')
    base_path = 'workplace/UCF50/'
    class_folder_path = os.path.join(base_path, selected_class_Name)
    video_files_names_list = os.listdir(class_folder_path)

    # Randomly select a video file name from the list of video files.
    selected_video_file_name = random.choice(video_files_names_list)

    # Initialize a VideoCapture object to read from the randomly selected video file.
    video_reader = cv2.VideoCapture(f'UCF50/{selected_class_Name}/{selected_video_file_name}')
    
    # Read the first frame from the video file.
    _, bgr_frame = video_reader.read()

    # Release the VideoCapture object to free up resources.
    video_reader.release()

    # Convert the frame from BGR to RGB format for display in Matplotlib.
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

    # Add the selected class name as text on the video frame.
    cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (125, 246, 55), 2)
    
    # Create a subplot in the figure and display the RGB frame in the subplot.
    plt.subplot(5, 4, counter)
    plt.imshow(rgb_frame)
    plt.axis('off')

In [ ]:
#Defining the Video Classification Constants and Variables
# Set the height and width of the video frames to 128x128 pixels.
IMAGE_HEIGHT, IMAGE_WIDTH = 128, 128

# Set the sequence length, which represents the number of frames per video sequence.
SEQUENCE_LENGTH = 15

# Define the list of classes that will be used for training the model.
SELECTED_CLASSES = ['Kayaking', 'Basketball', 'JumpRope']
#SELECTED_CLASSES = ['Kayaking', 'Basketball', 'JumpRope', 'Diving', 'HorseRace', 'PullUps','MilitaryParade']
#SELECTED_CLASSES = ['BenchPress', 'PullUps', 'PushUps'] # Special classes list


# Set the path to the dataset directory, which contains the UCF50 dataset.
DATASET_DIR = 'workspace/UCF50'    


In [ ]:
# Initialize MediaPipe Pose and Drawing utilities.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize the MediaPipe Pose model.
pose = mp_pose.Pose()


def preprocess_landmarks(landmarks):
    # Normalize the landmarks to be relative to the hip center
    hip_center = np.mean([landmarks[23], landmarks[24]], axis=0)
    normalized_landmarks = landmarks - hip_center
    
    # Flatten the landmarks array
    flattened = normalized_landmarks.flatten()
    
    # Normalize the flattened array
    return (flattened - np.mean(flattened)) / np.std(flattened)

In [ ]:
#Save and Load Pre-Processed Data
def save_extracted_data(frames, labels, file_name):
    if not os.path.exists('pre-processed'):
        os.makedirs('pre-processed')
    np.savez_compressed(os.path.join('pre-processed', file_name), frames=frames, labels=labels)

def load_preprocessed_data(file_name):
    data = np.load(os.path.join('pre-processed', file_name))
    return data['frames'], data['labels']

In [ ]:
#Preprocess Landmarks
def preprocess_landmarks(landmarks):
    # Normalize the landmarks to be relative to the hip center
    hip_center = np.mean([landmarks[23], landmarks[24]], axis=0)
    normalized_landmarks = landmarks - hip_center
    
    # Flatten the landmarks array
    flattened = normalized_landmarks.flatten()
    
    # Normalize the flattened array
    return (flattened - np.mean(flattened)) / np.std(flattened)


In [ ]:
# Function to extract frames from a video
def extract_frames(video_path):
    # Initialize empty lists to store video frames and poses.
    frames_list = []
    pose_list = []
    
    # Open the video file for reading using OpenCV's VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)
    
    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate the interval between frames to be added to the frames list.
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)

    # Loop through the specified number of frames (SEQUENCE_LENGTH).
    for frame_counter in range(SEQUENCE_LENGTH):
        
        # Set the position of the video to the appropriate frame.
        # This allows us to extract non-consecutive frames from the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        
        # Read the frame from the current position in the video.
        # 'success' will be True if the frame is read successfully.
        # 'frame' contains the frame data (an image in the form of a NumPy array).
        success, frame = video_reader.read()
        
        # If the frame cannot be read successfully (e.g., end of video reached), exit the loop.
        if not success:
            break

        # Resize the frame to a fixed height and width (IMAGE_HEIGHT, IMAGE_WIDTH).
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the pixel values of the resized frame to the range [0, 1].
        # Divide by 255 to convert pixel values from [0, 255] to [0, 1].
        normalized_frame = resized_frame / 255.0

        # Convert the frame from BGR to RGB color format.
        frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect the pose landmarks.
        results = pose.process(frame_rgb)
        
        # Append the normalized frame to the frames_list.
        frames_list.append(normalized_frame)

        # Extract the pose landmarks from the frame.
        if results.pose_landmarks:
            
            # Get the pose landmarks from the results.
            landmarks = results.pose_landmarks.landmark

            # Convert the landmarks to a NumPy array.
            pose_landmarks = np.array([(lm.x, lm.y, lm.z) for lm in landmarks])

            # Preprocess the pose landmarks to normalize the coordinates.
            preprocessed_landmarks = preprocess_landmarks(pose_landmarks)
            
            # Append the preprocessed landmarks to the pose_list.
            pose_list.append(preprocessed_landmarks)

            # Drawing the pose landmarks.
            if frame_counter == 0:
                
                # Draw the pose landmarks on the first frame only.
                mp_drawing.draw_landmarks(resized_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                # Save the frame with the pose landmarks drawn as an image file.
                cv2.imwrite('sample_frame_with_pose.jpg', resized_frame)            
        else:
            # If no pose landmarks are detected, append a list of zeros to the pose_list.
            pose_list.append(np.zeros(99))  # 33 landmarks * 3 coordinates

    # Release the VideoCapture object to free up resources.
    video_reader.release()
    
    # Return the list of normalized frames and poses extracted from the video.
    return frames_list, pose_list

In [ ]:
#Save and Load Dataset
def save_dataset(frame_features, pose_features, labels, filename='dataset-3-special.pkl'):
    """Save the extracted features and labels to a file."""
    with open(filename, 'wb') as f:
        pickle.dump((frame_features, pose_features, labels), f)
    print(f"Dataset saved to {filename}")

def load_dataset(filename='dataset-3-special.pkl'):
    """Load the extracted features and labels from a file."""
    with open(filename, 'rb') as f:
        frame_features, pose_features, labels = pickle.load(f)
    print(f"Dataset loaded from {filename}")
    return frame_features, pose_features, labels



In [ ]:
#Extract Video Data
def extract_video_data(save=True, filename='dataset-3-special.pkl'):
    if os.path.exists(filename):
        print(f"Loading existing dataset from {filename}")
        return load_dataset(filename)
    extracted_frames = []
    extracted_poses = []
    extracted_labels = []


    
    for class_index, class_name in enumerate(SELECTED_CLASSES):
        print(f'Data Extracting of Class: {class_name}')

        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            frames, poses = extract_frames(video_file_path)
            if len(frames) == SEQUENCE_LENGTH:
                extracted_frames.append(frames)
                extracted_poses.append(poses)
                extracted_labels.append(class_index)

    #return np.array(extracted_frames), np.array(extracted_poses), np.array(extracted_labels)
    frame_features = np.array(extracted_frames)
    pose_features = np.array(extracted_poses)
    labels = np.array(extracted_labels)

    if save:
        save_dataset(frame_features, pose_features, labels, filename)

    return frame_features, pose_features, labels

# Extract features and labels
#frame_features, pose_features, labels = extract_video_data()
frame_features, pose_features, labels = extract_video_data(save=True)

# One-hot encode labels
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
# Split the data
X_train_frames, X_test_frames, X_train_poses, X_test_poses, y_train, y_test = train_test_split(
    frame_features, pose_features, one_hot_encoded_labels, test_size=0.2, random_state=42)

# Print the shapes of the training features and labels after the split.
X_train_frames.shape, X_train_poses.shape, y_train.shape

## Print the shapes of the test features and labels after the split.
X_test_frames.shape, X_test_poses.shape, y_test.shape

In [ ]:
#Optuna Optimization
def objective(trial):
    # Define hyperparameters to be optimized
    num_conv_layers = trial.suggest_int('num_conv_layers', 1, 4)
    #num_filters = trial.suggest_categorical('num_filters', (16, 256))
    num_filters = trial.suggest_int('num_filters', 16, 256, step=16)
    kernel_size = trial.suggest_categorical('kernel_size', [(3, 3), (5, 5)])
    lstm_units = trial.suggest_int('lstm_units', 16, 256, step = 16)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    epochs = trial.suggest_int('epochs', 5, 50)
    batch_size = trial.suggest_int('batch_size', 8, 32, step = 16)

    # Build the combined model
    # Image input branch
    image_input = Input(shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    x1 = image_input
    for _ in range(num_conv_layers):
        x1 = TimeDistributed(Conv2D(num_filters, kernel_size, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))(x1)
        x1 = TimeDistributed(MaxPooling2D((2, 2)))(x1)
        x1 = TimeDistributed(Dropout(dropout_rate))(x1)
    x1 = TimeDistributed(Flatten())(x1)
    x1 = LSTM(lstm_units, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x1)

    # Pose input branch
    pose_input = Input(shape=(SEQUENCE_LENGTH, 99))
    x2 = LSTM(lstm_units, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01))(pose_input)
    x2 = LSTM(16)(x2)
    
    # Combine the two branches
    combined = concatenate([x1, x2])
    x = Dense(64, activation='relu')(combined)
    x = Dropout(dropout_rate)(x)
    output = Dense(len(SELECTED_CLASSES), activation='softmax')(x)

    model = Model(inputs=[image_input, pose_input], outputs=output)

    # Compile the model with weight decay
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Early stopping and learning rate reduction callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

    # Train the model
    history = model.fit([X_train_frames, X_train_poses], y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2,
                        callbacks=[TFKerasPruningCallback(trial, 'val_accuracy'), early_stopping, reduce_lr], verbose=0)

    # Evaluate the model
    score = model.evaluate([X_test_frames, X_test_poses], y_test, verbose=0)
    return score[1]  # Return validation accuracy


In [ ]:
# Define the pruner with min_resource
pruner = SuccessiveHalvingPruner(min_resource=10, reduction_factor=3)

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', pruner=pruner)
study.optimize(objective, n_trials=50)

In [ ]:
# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)

In [ ]:
#Create LRCN Model
def create_best_LRCN_model(best_params):
    best_params = best_params
    
    # Image input branch
    image_input = Input(shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3), name='image_input')
    x1 = image_input
    for _ in range(best_params['num_conv_layers']):
        x1 = TimeDistributed(Conv2D(best_params['num_filters'], best_params['kernel_size'], padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))(x1)
        x1 = TimeDistributed(MaxPooling2D((2, 2)))(x1)
        x1 = TimeDistributed(Dropout(best_params['dropout_rate']))(x1)
    x1 = TimeDistributed(Flatten())(x1)
    x1 = LSTM(best_params['lstm_units'], kernel_regularizer=tf.keras.regularizers.l2(0.01))(x1)

    # Pose input branch
    pose_input = Input(shape=(SEQUENCE_LENGTH, 99), name='pose_input')  # POSE_FEATURES should be defined based on your pose data
    x2 = LSTM(best_params['lstm_units'], return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01))(pose_input)
    x2 = LSTM(16)(x2)

    # Combine the two branches
    combined = concatenate([x1, x2])
    x = Dense(64, activation='relu')(combined)
    x = Dropout(best_params['dropout_rate'])(x)
    output = Dense(len(SELECTED_CLASSES), activation='softmax')(x)

    model = Model(inputs=[image_input, pose_input], outputs=output)

    # Compile the model with weight decay
    optimizer = tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'])
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [ ]:
#Create Model
model = create_best_LRCN_model(best_params)
#model = create_combined_model()
# Display the success message.
print("Model Created Successfully!")

# Count the number of layers
num_layers = len(model.layers)
print(f'The model has {num_layers} layers.')

In [ ]:
# Create and compile the model
learning_rate = best_params['learning_rate']

optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the custom optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    [X_train_frames, X_train_poses], y_train,
    validation_data=([X_test_frames, X_test_poses], y_test),
    epochs=10,
    batch_size=8,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        
    ]
)

In [ ]:
#Save Model
model.save('best_model_fusion-3-special.h5')

In [ ]:
# Evaluate the trained model.
# X_train_frames, X_test_frames, X_train_poses, X_test_poses, y_train, y_test
model_evaluation_history = model.evaluate([X_test_frames, X_test_poses], y_test)

In [ ]:
# Evaluate the model on the test set
all_predictions = []
all_labels = []

# Use the existing test data
for frames, poses, labels in tqdm(zip(X_test_frames, X_test_poses, y_test), desc="Testing", total=len(X_test_frames)):
    outputs = model.predict([np.expand_dims(frames, axis=0), np.expand_dims(poses, axis=0)])
    predicted = np.argmax(outputs, axis=1)
    all_predictions.extend(predicted)
    all_labels.extend(np.argmax(labels.reshape(1, -1), axis=1))

In [ ]:
#Plot Training Curves
def plot_training_curves(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(122)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.savefig('training_curves.png')
    plt.close()
    
# Plot training curves
plot_training_curves(history)

In [ ]:
#Plot Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_percent, annot=True, fmt='.1f', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix (Percentages)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix_percent.png')
    plt.close()

# Plot confusion matrix
plot_confusion_matrix(all_labels, all_predictions, SELECTED_CLASSES)


In [ ]:
#Plot Classification Report
def plot_classification_report(y_true, y_pred, classes):
    report = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
    df = pd.DataFrame(report).transpose()
    df = df.drop(['accuracy', 'macro avg', 'weighted avg'])
    df = df.drop('support', axis=1)  # Remove the support column

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axis('off')
    table = ax.table(cellText=df.values.round(2),
                     rowLabels=df.index,
                     colLabels=df.columns,
                     cellLoc='center',
                     loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)

    for i, key in enumerate(df.columns):
        cell = table[0, i]
        cell.set_text_props(weight='bold', color='white')
        cell.set_facecolor('#4C72B0')

    plt.title('Classification Report', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('classification_report.png', dpi=300, bbox_inches='tight')
    plt.close()

# Plot classification report
plot_classification_report(all_labels, all_predictions, SELECTED_CLASSES)


In [ ]:
#Plot ROC Curve
def plot_roc_curve(all_labels, all_predictions, SELECTED_CLASSES):
    y_true = label_binarize(all_labels, classes=range(len(SELECTED_CLASSES)))
    y_pred = label_binarize(all_predictions, classes=range(len(SELECTED_CLASSES)))

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(SELECTED_CLASSES)):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(12, 8))
    colors = plt.cm.get_cmap('Set1')(np.linspace(0, 1, len(SELECTED_CLASSES)))
    for i, color in zip(range(len(SELECTED_CLASSES)), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'ROC curve of {SELECTED_CLASSES[i]} (area = {roc_auc[i]:0.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right", fontsize='small')
    plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
    plt.close()

# Plot ROC curve
plot_roc_curve(all_labels, all_predictions, SELECTED_CLASSES)

In [ ]:
def predict_and_analyze_video(video_file_path, output_video_path, output_graph_path, model, SEQUENCE_LENGTH, SELECTED_CLASSES):
    # Suppress TensorFlow logging
    tf.get_logger().setLevel('ERROR')
    
    # Video setup
    video_reader = cv2.VideoCapture(video_file_path)
    fps = int(video_reader.get(cv2.CAP_PROP_FPS))
    frame_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 
                                   fps, (original_video_width, original_video_height))

    # Initialize variables for prediction and analysis
    frames_queue = deque(maxlen=SEQUENCE_LENGTH)
    pose_queue = deque(maxlen=SEQUENCE_LENGTH)
    frame_predictions = []
    class_probabilities = defaultdict(list)
    predicted_class_name = ''

    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    pose = mp_pose.Pose()

    # Process video frames
    with tqdm(total=frame_count, desc="Processing video") as pbar:
        while video_reader.isOpened():
            ok, frame = video_reader.read()
            if not ok:
                break

            # Preprocess the frame
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            resized_frame = cv2.resize(frame_rgb, (IMAGE_HEIGHT, IMAGE_WIDTH))
            normalized_frame = resized_frame / 255.0

            # Process the frame for pose detection
            results = pose.process(frame_rgb)

            if results.pose_landmarks:
                # Draw pose landmarks on the frame
                mp_drawing.draw_landmarks(
                    frame, 
                    results.pose_landmarks, 
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                )

                # Extract and preprocess landmarks for prediction
                landmarks = results.pose_landmarks.landmark
                pose_landmarks = np.array([(lm.x, lm.y, lm.z) for lm in landmarks])
                preprocessed_landmarks = preprocess_landmarks(pose_landmarks)
            else:
                preprocessed_landmarks = np.zeros(99)  # 33 landmarks * 3 coordinates

            frames_queue.append(normalized_frame)
            pose_queue.append(preprocessed_landmarks)

            if len(frames_queue) == SEQUENCE_LENGTH:
                # Prepare input for the model
                input_frames = np.expand_dims(np.array(frames_queue), axis=0)
                input_poses = np.expand_dims(np.array(pose_queue), axis=0)

                # Get predictions
                with tf.device('/GPU:0'):
                    outputs = model.predict([input_frames, input_poses], verbose=0)
                probabilities = outputs[0]
                predicted_class_index = np.argmax(probabilities)
                predicted_class_name = SELECTED_CLASSES[predicted_class_index]

                # Store predictions and probabilities
                frame_predictions.append(predicted_class_name)
                for i, class_name in enumerate(SELECTED_CLASSES):
                    class_probabilities[class_name].append(probabilities[i])
            else:
                frame_predictions.append(None)
                for class_name in SELECTED_CLASSES:
                    class_probabilities[class_name].append(0)

            # Write predicted class name on top of the frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame into the disk using the VideoWriter Object
            video_writer.write(frame)

            # Update progress bar
            pbar.update(1)

    video_reader.release()
    video_writer.release()
    pose.close()

    # Pad the beginning of predictions and probabilities
    pad_length = SEQUENCE_LENGTH - 1
    frame_predictions = [None] * pad_length + frame_predictions[pad_length:]
    for class_name in SELECTED_CLASSES:
        class_probabilities[class_name] = [0] * pad_length + class_probabilities[class_name][pad_length:]

    # Plot frame-by-frame results
    plot_frame_by_frame_results(frame_predictions, class_probabilities, fps, output_graph_path)

    return frame_predictions, class_probabilities, fps
#Plot Frame by Frame Results
def plot_frame_by_frame_results(frame_predictions, class_probabilities, fps, output_path):
    frame_count = len(frame_predictions)
    time_axis = np.arange(frame_count) / fps
    
    plt.figure(figsize=(15, 10))
    
    # Plot class probabilities
    plt.subplot(2, 1, 1)
    for class_name, probs in class_probabilities.items():
        plt.plot(time_axis, probs, label=class_name)
    plt.title("Class Probabilities Over Time")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Probability")
    plt.legend()
    plt.grid(True)
    
    # Plot predicted classes
    plt.subplot(2, 1, 2)
    unique_classes = list(set(frame_predictions) - {None})
    class_to_num = {cls: i for i, cls in enumerate(unique_classes)}
    numeric_predictions = [class_to_num[cls] if cls is not None else -1 for cls in frame_predictions]
    plt.scatter(time_axis, numeric_predictions, marker='.')
    plt.yticks(range(len(unique_classes)), unique_classes)
    plt.title("Predicted Class Over Time")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Predicted Class")
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

In [ ]:
#Process Test Video
def process_test_video(model, SEQUENCE_LENGTH, SELECTED_CLASSES):
    test_videos_directory = 'test_videos'
    video_title = 'pu3'  # Replace with the actual video title

    input_video_file_path = f'{test_videos_directory}/{video_title}.mp4'
    output_video_file_path = f'{test_videos_directory}/{video_title}-Output-SeqLen{SEQUENCE_LENGTH}.mp4'
    output_graph_path = f'{test_videos_directory}/{video_title}_frame_analysis.png'

    frame_predictions, class_probabilities, fps = predict_and_analyze_video(
        input_video_file_path, output_video_file_path, output_graph_path,
        model, SEQUENCE_LENGTH, SELECTED_CLASSES
    )

    print(f"Processed video saved to: {output_video_file_path}")
    print(f"Frame-by-frame analysis graph saved to: {output_graph_path}")

# Call this function after training your model
process_test_video(model, SEQUENCE_LENGTH, SELECTED_CLASSES)

In [ ]:
#EXTRA:
# Function to download videos from YouTube.
def download_yt_videos(yt_url_list, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    ydl_opts = {
        'outtmpl': os.path.join(save_dir, '%(title)s.%(ext)s'),
        'format': 'bestvideo+bestaudio/best',
        'merge_output_format': 'mp4'
    }
    
    for url in yt_url_list:
        try:
            with YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
                print(f"Downloaded: {url}")
        except Exception as e:
            print(f"Failed to download {url}: {e}")

yt_url_list = [
    'https://www.youtube.com/shorts/y7PBQ2fYbxY'
    
]
save_dir = 'test_data'
download_yt_videos(yt_url_list, save_dir)